In [1]:
import dataset_utils as du
import pandas as pd
import matplotlib.pyplot as plt
import metric_utils as mu
import als_model as als
import json
import pickle
%matplotlib inline

In [2]:
ratings = du.processRatings()
r_train, r_val = du.trainvalTransactions(ratings, border=0.80)
ds_train = du.makeTargetRating(r_train)
ds_val = du.makeTargetRating(r_val)
ds_val = ds_val[ds_val['y'] > 5]
print('Train len:', len(ds_train))
print('Val len:', len(ds_val))

Train len: 365087
Val len: 65505


/Users/u18277461/Documents/repos/rekko-challenge/dataset_utils.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['y'] = ds['rating']


In [11]:
best_model, best_score, _ = als.tuneALS(ds_train, ds_val, factors = [3, 4])
# f=3: MNAP@20: 0.029754151276486375
# f=4: MNAP@20: 0.031523156072250674
# f=5: MNAP@20: 0.0346529314836487
# f=6: MNAP@20: 0.03408368346022049
# f=7: MNAP@20: 0.033711182310522746
# f=8: MNAP@20: 0.03312585901787205
# f=10: MNAP@20: 0.032622348762988515
# f=20: MNAP@20: 0.030265564081234594
# f=30: MNAP@20: 0.029549124691366124

#
# BEST F=5

Test users done.
Elements filtered out from validation
Sparsity: 0.000544
Iteration 1 of 2


/Users/u18277461/Documents/repos/rekko-challenge/als_model.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_train['user_uid'] = tr_train['user_uid'].astype('category')
/Users/u18277461/Documents/repos/rekko-challenge/als_model.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_train['element_uid'] = tr_train['element_uid'].astype('category')



Model 1 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.029754151276486375
Best model updated
Iteration 2 of 2



Model 2 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.0315498237787785
Best model updated


In [14]:
# PREDICT & SUBMIT

In [3]:
test_users = du.processTestUsers()
ds_train_all = du.makeTargetRating(ratings)

final_model, _, t_result = als.tuneALS(ds_train_all, ds_val[:1], factors = [5], t_users=test_users)

Test users done.
Elements filtered out from validation
Sparsity: 0.000558
Iteration 1 of 1


/Users/u18277461/Documents/repos/rekko-challenge/als_model.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_train['user_uid'] = tr_train['user_uid'].astype('category')
/Users/u18277461/Documents/repos/rekko-challenge/als_model.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_train['element_uid'] = tr_train['element_uid'].astype('category')



Model 1 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.0
Best model updated


In [5]:
def saveSubmit(name, model, results):
    with open('submits/{0}.json'.format(name), 'w') as f:
        json.dump(t_result, f)
    print('Saved results {0}.json'.format(name))

    with open('submits/model_{0}.pkl'.format(name), 'wb') as f:
        pickle.dump(model, f)
    print('Saved model model_{0}.pkl'.format(name))

In [6]:
saveSubmit('als_rating_f5', final_model, t_result)

Saved results als_rating_f5.json
Saved model model_als_rating_f5.pkl
